# Task 4
In this exercise, you will implement a model to classify images. Each image belongs to one out of ten classes and is given as RGB image with 32x32 pixels. This task focuses on the creation of data lists to load your data and on the implementation of a custom dataset class.

**a)** Download the dataset to your hard disk and extract the files. The data is provided as train-test split and the labels of the respective images are given by the name of the parent folder. Download the CIFAR10 dataset from  https://www.kaggle.com/datasets/swaroopkml/cifar10-pngs-in-folders?resource=download or from https://owncloud.csl.uni-bremen.de/s/9mNnmeA7esyEpnC. The corresponding paper of the CIFAR10 dataset: *"Learning Multiple Layers of Features from Tiny Images", Alex Krizhevsky, 2009*.

**b)** Take the last 20 % of the images of each class and save them in a separate folder structure for validation, e.g.:  
*\user\data\cifar10\validate\airplane\4001.png  
\user\data\cifar10\validate\airplane\4002.png  
...  
\user\data\cifar10\validation\bird\4001.png  
...*  
This leads to the following datasplit for each class: 4,000/1,000/1,000 images for training/validation/testing respectively.

**c)** Create one data file list for the training data, one for the validation data, and one for the test data. Each list contains the paths to the different images and the respective labels. You can save the lists as .txt files. You will use these lists in your custom dataset class. One possible example is given in the following.  
The training_list.txt contains the respective training data paths and the labels, separated by white spaces:  
*\user\data\cifar10\train\airplane\0001.png airplane  
\user\data\cifar10\train\airplane\0002.png airplane  
...  
\user\data\cifar10\train\bird\0001.png bird  
...*  

The validation_list.txt contains the respective validation data paths and the labels, separated by white spaces.  
*\user\data\cifar10\validate\airplane\4001.png airplane  
\user\data\cifar10\validate\airplane\4002.png airplane  
...  
\user\data\cifar10\validate\bird\4001.png bird  
...*  

The test_list.txt contains the respective test data paths and the labels, separated by white spaces.  
*\user\data\cifar10\test\airplane\0001.png airplane  
\user\data\cifar10\test\airplane\0002.png airplane  
...  
\user\data\cifar10\test\bird\0001.png bird  
...*  

**d)** Write a custom dataset class to define how to load and prepare the data. You can use the PyTorch dataloader class to interface your custom dataset class.  
**HINT1:** Use the io method of the scikit-image package to load your data. You need to install the scikit-image package to your AML_Tut conda environment.  
**HINT2:** You can install and use the cv2 package (opencv) to display RGB images.

**e)** Complete the script to create, train and test a classification model. It is not required to develope a complex model with high classification accuracy. This tasks aims to provide you withe hands-on experience on how to set up, train, and test a machine learning system for a specific task.


In [1]:
# Advanced Machine Learning Tutorials
# Excercise Sheet 1 - Task 5
# Authors: Marvin Borsdorf, Yale Hartmann
# Cognitive Systems Lab, University of Bremen, Germany
# Last edit: 2021/04/26

import os
import cv2
import torch as th
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skimage import io
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
root_dir = r'C:\Users\m-gre\Documents\Advanced Machine Learning\uni-bremen-aml-course\Mario\Exercise_1'
tr_dir = os.path.join(root_dir, 'cifar10', 'train')
cv_dir = os.path.join(root_dir, 'cifar10', 'validate')  
tt_dir = os.path.join(root_dir, 'cifar10', 'test')

In [3]:
# Create datalist from directory for each subset
tr_file_list = []
cv_file_list = []
tt_file_list = []

# Training
for path, subdirs, files in os.walk(tr_dir):
    tr_file_list.extend([os.path.join(path, s) + " " + os.path.basename(path) for s in files])
        
# Validation
for path, subdirs, files in os.walk(cv_dir):
    cv_file_list.extend([os.path.join(path, s) + " " + os.path.basename(path) for s in files])

# Test
for path, subdirs, files in os.walk(tt_dir):
    tt_file_list.extend([os.path.join(path, s) + " " + os.path.basename(path) for s in files])

tr_file_list

['C:\\Users\\m-gre\\Documents\\Advanced Machine Learning\\uni-bremen-aml-course\\Mario\\Exercise_1\\cifar10\\train\\airplane\\0001.png airplane',
 'C:\\Users\\m-gre\\Documents\\Advanced Machine Learning\\uni-bremen-aml-course\\Mario\\Exercise_1\\cifar10\\train\\airplane\\0002.png airplane',
 'C:\\Users\\m-gre\\Documents\\Advanced Machine Learning\\uni-bremen-aml-course\\Mario\\Exercise_1\\cifar10\\train\\airplane\\0003.png airplane',
 'C:\\Users\\m-gre\\Documents\\Advanced Machine Learning\\uni-bremen-aml-course\\Mario\\Exercise_1\\cifar10\\train\\airplane\\0004.png airplane',
 'C:\\Users\\m-gre\\Documents\\Advanced Machine Learning\\uni-bremen-aml-course\\Mario\\Exercise_1\\cifar10\\train\\airplane\\0005.png airplane',
 'C:\\Users\\m-gre\\Documents\\Advanced Machine Learning\\uni-bremen-aml-course\\Mario\\Exercise_1\\cifar10\\train\\airplane\\0006.png airplane',
 'C:\\Users\\m-gre\\Documents\\Advanced Machine Learning\\uni-bremen-aml-course\\Mario\\Exercise_1\\cifar10\\train\\airplane

In [4]:
from torchvision import io

# Define custom dataset
class cifar10Dataset(Dataset):
    def __init__(self, datalist): 
        # You need access to the data file lists. Provide the list as parameter somehow...
        # Create the final splitted data and label lists
        # ...complete code here
        self.data, self.label = zip(*[item.rsplit(' ', 1) for item in datalist])
        self._init_data()
        
    def __len__(self):                      # Get total number of dataset samples
        # ...complete code here
        return len(self.label)

    def __getitem__(self, idx):             # Return an item
        # ...complete code here
        data = io.read_image(self.data[idx]).float() / 255.0 # Normalize Values to [0,1]
        return data, self.label[idx]
        
    def _init_data(self):                   # Initialize data
        # ...complete code here
        # Read file
        # This is also a good place to encode your string labels into integer labels for the model prediction (-> use a LabelEncoder)
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(self.label)
        self.label = self.label_encoder.transform(self.label)

In [12]:
# Define model (neural network)
class BaselineModel(nn.Module):
    def __init__(self):
        # ...complete code here
        super(BaselineModel, self).__init__()
        self.model = nn.Sequential(
            nn.LazyConv2d(32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.LazyConv2d(32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.LazyConv2d(64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.LazyConv2d(64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.LazyConv2d(128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.LazyConv2d(128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten(),
            nn.LazyLinear(128),
            nn.ReLU(),

            nn.LazyLinear(10),
        )

    def forward(self, x):
        # ...complete code
        return self.model(x)

In [13]:
# Define loss
criterion = nn.CrossEntropyLoss()  # Applies log_softmax and negative log likelihood loss

# Create datasets
training_dataset = cifar10Dataset(tr_file_list)
validation_dataset = cifar10Dataset(cv_file_list)

# PyTorch dataloader
training_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=True)

# Create model
model = BaselineModel()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001) # You may change the optimizer, the learning rate, and other hyperparameters

c:\Users\m-gre\miniconda3\envs\AML_Tut\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [14]:
# Define training step
def training(model, data_loader, optimizer):
    training_losses = []
    training_correct = []
    
    model.train()                   # Important
    for data in tqdm(data_loader):
        inputs, labels = data
        
        # ...complete code here
        # Zero the last gradients
        optimizer.zero_grad()
        # Do prediction
        predictions = model(inputs)
        
        labels = labels.long()
        
        # ...complete code here
        # Calculate loss
        loss = criterion(predictions, labels)
        # Do backpropagation and calculate gradients
        loss.backward()
        # Update weights
        optimizer.step()
        
        training_losses.append(loss.item())
        correct = (th.argmax(predictions, dim=1) == labels) # Check for correct classification
        training_correct.append(sum(correct).item())
    
    training_accuracy = 100*(np.sum(training_correct)/len(data_loader.dataset))
    return np.mean(training_losses), training_accuracy

In [15]:
# Define validation step
def validation(model, data_loader, optimizer):
    validation_losses = []
    validation_correct = []
    
    model.eval()                        # Important to disable parts such as dropout, batch norm, etc.
    with th.no_grad():                  # No gradients
        for data in tqdm(data_loader):
            inputs, labels = data
            # ...complete code here
            # Do prediction
            predictions = model(inputs)
            
            labels = labels.long()
            
            # ...complete code here
            # Calculate loss
            loss = criterion(predictions, labels)
            
            validation_losses.append(loss.item())
            correct = (th.argmax(predictions, dim=1) == labels) # Check for correct classification
            validation_correct.append(sum(correct).item())
    
    validation_accuracy = 100*(np.sum(validation_correct)/len(data_loader.dataset))
    return np.mean(validation_losses), validation_accuracy

In [16]:
# Run the training and the validation
epochs = 30
print("Train on %d samples." % len(training_loader.dataset))
print("Validate on %d samples." % len(validation_loader.dataset))

for e in range(epochs):
    print("###### Epoch %d ######" %(e+1))
    tr_loss, tr_acc = training(model, training_loader, optimizer)
    print("Training loss: %.5f. Training accuracy: %.2f %%." % (tr_loss, tr_acc))
    val_loss, val_acc = validation(model, validation_loader, optimizer)
    print("Validation loss: %.5f. Validation accuracy: %.2f %%." % (val_loss, val_acc))

Train on 40000 samples.
Validate on 10000 samples.
###### Epoch 1 ######


100%|██████████| 1250/1250 [02:49<00:00,  7.39it/s]


Training loss: 1.67483. Training accuracy: 37.94 %.


100%|██████████| 313/313 [00:17<00:00, 18.07it/s]


Validation loss: 1.36039. Validation accuracy: 49.77 %.
###### Epoch 2 ######


...

Validation loss: 1.66502. Validation accuracy: 72.92 %.
###### Epoch 30 ######


100%|██████████| 1250/1250 [02:06<00:00,  9.88it/s]


Training loss: 0.12011. Training accuracy: 96.03 %.


100%|██████████| 313/313 [00:11<00:00, 27.47it/s]

Validation loss: 1.73957. Validation accuracy: 73.65 %.


In [17]:
# Save model and optimizer state dictionaries
th.save(model.state_dict(), os.path.join(root_dir, 'model_state_dict.pt'))   
th.save(optimizer.state_dict(), os.path.join(root_dir, 'optimizer_state_dict.pt'))

In [18]:
# Test model on test input
model.eval()    # Important to disable parts such as dropout, batch norm, etc.
test_dataset = cifar10Dataset(tt_file_list)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
m1_losses = []
m1_correct = []

print("Test on %d samples." % len(test_loader.dataset))

for data in tqdm(test_loader):
        _input, label = data
        label = label.long()

        # ...complete code here
        # Do prediction
        pred1 = model(_input)
        # Calculate loss
        loss = criterion(pred1, label)
        
        m1_losses.append(loss.item())
        correct = (th.argmax(pred1, dim=1) == label) # Check for correct classification
        m1_correct.append(sum(correct).item())
        
        # Uncomment to print image and prediction
        #print("Model 1 prediction: %d -> %s" %(th.argmax(pred1, dim=1), test_dataset.label_encoder.inverse_transform(th.argmax(pred1, dim=1))))
        #plt.imshow(cv2.cvtColor(th.squeeze(_input).numpy().reshape(32, 32, 3), cv2.COLOR_BGR2RGB))
        #plt.show()

# Calculate final scores
m1_loss = np.mean(m1_losses)
m1_acc = 100*(np.sum(m1_correct)/len(test_loader.dataset))

print("Model 1")
print("Test loss: %.5f. Test accuracy: %.2f %%." % (m1_loss, m1_acc))


Test on 10000 samples.


  0%|          | 0/313 [00:00<?, ?it/s]

100%|██████████| 313/313 [00:11<00:00, 27.68it/s]

Model 1
Test loss: 1.79669. Test accuracy: 71.89 %.


Solution could get accuracy of 96%